In [ ]:
# データの永続化なしversion（メモリに保存）

import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import os
import datetime
import time

def news():
    #nikki速報
    sokuhou = "https://www.nikkei.com/news/category/"
    response = requests.get(sokuhou)
    html = response.text
    soup = BeautifulSoup(html,"html.parser")

    url = "https://www.nikkei.com"
    result = []

    # #日経新聞のトップ記事を取得
    info = soup.find_all(class_="main_m1jop52u")

    for i in info:
        info = i.find_all("article")
        for j in info:
            info = j.find_all("a")
            info = info[0]
            title = info.text
            link_url = url + info.get('href')
            result.append([title,link_url])

    result = result
    return result


def list_diff(result, last_result):
    diff_list = []
    for tmp in result:
        if tmp not in last_result:
            diff_list.append(tmp)
    return diff_list


def send_to_line(diff_list):
    if diff_list != []:
        now = datetime.datetime.now()
        now = now.strftime("%Y/%m/%d %H:%M:%S")
        text = now + '\n'
        for tmp in diff_list:
            text += tmp[0] + '\n' + tmp[1] + '\n'

        # LINE Notifyのアクセストークンを設定
        token = "xxxxxxxxxxxxxxxxxxxxxxxxx" 
        # APIエンドポイントのURLを定義
        line_url = "https://notify-api.line.me/api/notify"
        # HTTPリクエストヘッダーの設定 
        headers = {"Authorization" : "Bearer "+ token}
        # 送信するメッセージの設定
        message =  (text)
        # ペイロードの設定
        payload = {"message" :  message}
        # POSTリクエストの使用 
        r = requests.post(line_url, headers = headers, params=payload)

# main
if __name__ == '__main__':
    last_result = []
    while True:
        # 初回実行時の初期化
        if not last_result:
            result = news()
            last_result = result
        else:
            result = news()
            diff_list = list_diff(result, last_result)
            send_to_line(diff_list)
            last_result = result  # 最新の結果を保存
        time.sleep(3)
        

In [ ]:
#データの永続化version（csv保存）

import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import os
import datetime
import time

def news():
    #nikki速報
    sokuhou = "https://www.nikkei.com/news/category/"
    response = requests.get(sokuhou)
    html = response.text
    soup = BeautifulSoup(html,"html.parser")

    url = "https://www.nikkei.com"
    result = []

    # #日経新聞のトップ記事を取得
    info = soup.find_all(class_="main_m1jop52u")

    for i in info:
        info = i.find_all("article")
        for j in info:
            info = j.find_all("a")
            info = info[0]
            title = info.text
            link_url = url + info.get('href')
            result.append([title,link_url])

    result = result
    return result

def output_csv(result):
    #last_loh.csvとファイルディレクトリを開いているとエラーになる
    with open('last_log.csv', 'w', newline='',encoding='utf-8-sig') as file:
        headers = ['Title', 'URL']
        writer = csv.writer(file)
        writer.writerow(headers)
        for row in result:
            writer.writerow(row)

def read_csv():
    if not os.path.exists('last_log.csv'):
        result = news() # 新規のニュースを取得
        output_csv(result) # 保存データを更新
        raise Exception('新規ファイルを作成しました。')
    if os.path.getsize('last_log.csv') == 0:
        raise Exception('ファイルの中身が空です。')
    last_result = pd.read_csv('last_log.csv', header=None).values.tolist() #DataFrame から List への変換
    return last_result

def list_diff(result, last_result):
    diff_list = []
    for tmp in result:
        if tmp not in last_result:
            diff_list.append(tmp)
    return diff_list

def send_to_line(diff_list):
    if diff_list != []:
        now = datetime.datetime.now()
        now = now.strftime("%Y/%m/%d %H:%M:%S")
        text = now + '\n'
        for tmp in diff_list:
            text += tmp[0] + '\n' + tmp[1] + '\n'

        # LINE Notifyのアクセストークンを設定
        token = "xxxxxxxxxxxxxxxxxxxxxxxxx" 
        # APIエンドポイントのURLを定義
        line_url = "https://notify-api.line.me/api/notify"
        # HTTPリクエストヘッダーの設定 
        headers = {"Authorization" : "Bearer "+ token}
        # 送信するメッセージの設定
        message =  (text)
        # ペイロードの設定
        payload = {"message" :  message}
        # POSTリクエストの使用 
        r = requests.post(line_url, headers = headers, params=payload)

# main
if __name__ == '__main__':
    while True:
        last_result = read_csv() # 以前の保存データを参照
        result = news() # 新規のニュースを取得
        output_csv(result) # 保存データを更新
        diff_list = list_diff(result, last_result) # 以前のデータと新規のニュースを照合
        send_to_line(diff_list)
        time.sleep(5)